In [1]:
import sys
sys.path.append("../src/")

In [2]:
%load_ext autoreload
%autoreload 2
import data_utils as du

In [3]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  


In [4]:
data = du.get_wiki_data(5)

INFO:absl:No config specified, defaulting to first: wiki40b/en
INFO:absl:Load dataset info from gs://tfds-data/datasets/wiki40b/en/1.3.0
INFO:absl:Field info.config_name from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.config_description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.splits from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.module_name from disk and from code do not match. Keeping the one from code.
INFO:absl:Reusing dataset wiki40b (gs://tfds-data/datasets/wiki40b/en/1.3.0)
INFO:absl:Constructing tf.data.Dataset for split train[:5], from gs://tfds-data/datasets/wiki40b/en/1.3.0


# Given 1 document

* split doc into paragraphs
* filter paragraph by length
* encode each paragraph
* serialize output

In [9]:
du.get_paragraphs(data[1])

['\nKasturba Road is a street in Bangalore, the capital of Karnataka, India, which is connected to M G Road to the north and J C Road to the south. Some important landmarks situated along Kasturba Road are Sree Kanteerava Stadium, Kanteerava Indoor Stadium, Cubbon Park, Government Museum, Venkatappa Art Gallery, Visvesvaraya Industrial and Technological Museum and UB City. A 600-year-old Ganesha temple is also situated on Kasturba Road.\nIt was earlier known as Sydney Road.\nOther important landmarks close to the road are Karnataka High Court, Vidhana Soudha and Chinnaswamy Stadium.']

In [67]:
reprs = du.srl_paragraphs_batched(du.get_paragraphs(data[1]))

In [64]:
# Use cPickle
du.serialize_reprs(du.srl_paragraphs_batched(du.get_paragraphs(data[1])), "/tmp/test.pkl")

In [66]:
import pickle
with open("/tmp/test.pkl", "rb") as f:
    unpickler = pickle.Unpickler(f)
    test = unpickler.load() 
print(test)

('Kasturba Road is a street in Bangalore, the capital of Karnataka, India, which is connected to M G Road to the north and J C Road to the south.', [[('V', [6]), ('ARG1', [1, 2, 3, 4, 5]), ('ARG2', [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35])], [('V', [20])], [('V', [21]), ('ARG1', [7, 8, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35])]])


# Parallelize one worker
* batchify over multiple paragraphs
* write all outputs to single file

# shard data
* simply take a parameter for what slice of data to operate over
* check all pylint bugs

# Benchmark
* time execution of 1 paragraph on cpu vs gpu
* on best setup, optimize batch size
* do calculation and choose appropriate sharing

In [ ]:
paragraphs = []
for d in data:
    text = d['text'].numpy().decode('utf-8')
    for para in text.split("_START_"):
        if not para.startswith("PARAGRAPH_"):
            continue
        proced_text = du.preproc_text(para)
        if len(proced_text) < 500:
            continue
        paragraphs.append(proced_text)
        # print(proced_text)
        # print(len(proced_text))
        
        #print("~"*20)
    #print("#"*20)
print(len(paragraphs))

In [ ]:
print(paragraphs[100])

# Format
* We need to keep both paragraph level and document level examples together.
* We can train consecutively any paragraph but also we can train any document with multiple paragraphs

## First try
* Split by paragraph
* Filter for paragraph length (above some length)
* We get roughly 1 paragraph over 1000 characters per document. That's not great.
* 789 / 500 docs with cutoff at 500 characters

# TODO
* Upgrade Spacy en_core_web_sm to latest
* fix data encoder error

In [ ]:
reprs = du.encode_data_srl_only(data[:10])

In [ ]:
import sys

In [ ]:
sys.getsizeof(data[:10])

In [ ]:
sys.getsizeof(reprs)

In [ ]:
len(reprs)

In [ ]:
reprs[0]

In [ ]:
from collections import defaultdict



In [ ]:
def encode_data_srl_only(data):
    nlp = du.create_nlp()
    tokenizer = du.BertTokenizer.from_pretrained('bert-base-uncased')
    predictor = du.make_allennlp()
    reprs = []
    for ex_idx, d in enumerate(data):
        text = d['text'].numpy().decode('UTF-8').encode('ascii', 'ignore').decode('UTF-8')
        for para in text.split("_START_"):
            if not para.startswith("PARAGRAPH_"):
                continue
            example = du.preproc_text(para)
            if len(example) < 500:
                continue
            doc = nlp(example)
            for sent in doc.sents:
                span = sent.text
                lm_tokenized_input = tokenizer(span, return_tensors='pt')
                srl_parse = predictor.predict(sentence=span)
                token_strs = tokenizer.convert_ids_to_tokens(lm_tokenized_input['input_ids'].numpy()[0])
                try:
                    word2veclist = new_map_words_to_vectors(srl_parse['words'], token_strs)
                except du.NoStopException as e:
                    continue
                except Exception as e:
                    print("Index", ex_idx)
                    print(e)
                    print(token_strs)
                    print(srl_parse['words'])
                    raise e
                sent_repr = build_repr(srl_parse['verbs'], word2veclist)
                reprs.append((span, sent_repr))
    return reprs

In [ ]:
reprs = encode_data_srl_only(data[:2])
print(reprs)

In [ ]:
reprs[0]

In [ ]:
tokenizer = du.BertTokenizer.from_pretrained('bert-base-uncased')
span = '\nClayton was born in Mobile in south Alabama, but he was reared in Alexandria, Virginia.'
lm_tokenized_input = tokenizer(span, return_tensors='pt')


In [ ]:
lm_tokenized_input

# Fix parsing errors
* fix parser
* update spacy

In [ ]:
data = du.get_wiki_data(500)

In [ ]:
reprs = encode_data_srl_only(data[27:])

In [ ]:
reprs[33]

In [ ]:
predictor = du.make_allennlp()

In [ ]:
predictor.predict(sentence='\nClayton was born in Mobile in south Alabama, but he was reared in Alexandria, Virginia.'
)

In [ ]:
for ex_idx, d in enumerate(data[:1]):
    text = d['text'].numpy().decode('utf-8')
    for para in text.split("_START_"):
        if not para.startswith("PARAGRAPH_"):
            continue
        example = du.preproc_text(para)
        if len(example) < 500:
            continue
        doc = nlp(example)
        for sent in doc.sents:
            span = sent.text
            lm_tokenized_input = tokenizer(span, return_tensors='pt')
            srl_parse = predictor.predict(sentence=span)
            token_strs = tokenizer.convert_ids_to_tokens(lm_tokenized_input['input_ids'].numpy()[0])
            word2veclist = new_map_words_to_vectors(srl_parse['words'], token_strs)

In [ ]:
data[6]['text'].numpy().decode('UTF-8').encode('ascii', 'ignore').decode('UTF-8').split("_START_")# .decode("utf-8")

In [ ]:
type(data[6]['text'].numpy())

In [68]:
import spacy

In [69]:
spacy.__version__

'2.3.2'

In [71]:
long_str = """
The domains wikipedia.com and wikipedia.org were registered on January 12, 2001,[22] and January 13, 2001,[23] respectively, and Wikipedia was launched on January 15, 2001,[14] as a single English-language edition at www.wikipedia.com,[24] and announced by Sanger on the Nupedia mailing list.[18] Wikipedia's policy of "neutral point-of-view"[25] was codified in its first few months. Otherwise, there were relatively few rules initially and Wikipedia operated independently of Nupedia.[18] Originally, Bomis intended to make Wikipedia a business for profit.[26]


The Wikipedia home page on December 17, 2001
Wikipedia gained early contributors from Nupedia, Slashdot postings, and web search engine indexing. Language editions were also created, with a total of 161 by the end of 2004.[27] Nupedia and Wikipedia coexisted until the former's servers were taken down permanently in 2003, and its text was incorporated into Wikipedia. The English Wikipedia passed the mark of two million articles on September 9, 2007, making it the largest encyclopedia ever assembled, surpassing the Yongle Encyclopedia made during the Ming Dynasty in 1408, which had held the record for almost 600 years.[28]

Citing fears of commercial advertising and lack of control in Wikipedia, users of the Spanish Wikipedia forked from Wikipedia to create the Enciclopedia Libre in February 2002.[29] Wales then announced that Wikipedia would not display advertisements, and changed Wikipedia's domain from wikipedia.com to wikipedia.org.[30][31]

Though the English Wikipedia reached three million articles in August 2009, the growth of the edition, in terms of the numbers of new articles and of contributors, appears to have peaked around early 2007.[32] Around 1,800 articles were added daily to the encyclopedia in 2006; by 2013 that average was roughly 800.[33] A team at the Palo Alto Research Center attributed this slowing of growth to the project's increasing exclusivity and resistance to change.[34] Others suggest that the growth is flattening naturally because articles that could be called "low-hanging fruit"—topics that clearly merit an article—have already been created and built up extensively.[35][36][37]


File:Wikipedia Edit 2014.webm
A promotional video of the Wikimedia Foundation that encourages viewers to edit Wikipedia, mostly reviewing 2014 via Wikipedia content
In November 2009, a researcher at the Rey Juan Carlos University in Madrid found that the English Wikipedia had lost 49,000 editors during the first three months of 2009; in comparison, the project lost only 4,900 editors during the same period in 2008.[38][39] The Wall Street Journal cited the array of rules applied to editing and disputes related to such content among the reasons for this trend.[40] Wales disputed these claims in 2009, denying the decline and questioning the methodology of the study.[41] Two years later, in 2011, Wales acknowledged the presence of a slight decline, noting a decrease from "a little more than 36,000 writers" in June 2010 to 35,800 in June 2011. In the same interview, Wales also claimed the number of editors was "stable and sustainable".[42] A 2013 article titled "The Decline of Wikipedia" in MIT Technology Review questioned this claim. The article revealed that since 2007, Wikipedia had lost a third of its volunteer editors, and those still there have focused increasingly on minutiae.[43] In July 2012, The Atlantic reported that the number of administrators is also in decline.[44] In the November 25, 2013, issue of New York magazine, Katherine Ward stated "Wikipedia, the sixth-most-used website, is facing an internal crisis".[
"""
x = tokenizer(long_str, return_tensors="pt")

In [74]:
len(x['input_ids'][0])

789